<a href="https://colab.research.google.com/github/dauparas/tensorflow_examples/blob/master/VAE_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Simple template for the Variational Autoencoder Model (VAE):

https://arxiv.org/pdf/1312.6114.pdf

In [1]:
#@title Install { display-mode: "form" }
TF_Installation = 'TF2 Nightly (GPU)' #@param ['TF2 Nightly (GPU)', 'TF2 Stable (GPU)', 'TF1 Nightly (GPU)', 'TF1 Stable (GPU)','System']

if TF_Installation == 'TF2 Nightly (GPU)':
  !pip install -q --upgrade tf-nightly-gpu-2.0-preview
  print('Installation of `tf-nightly-gpu-2.0-preview` complete.')
elif TF_Installation == 'TF2 Stable (GPU)':
  !pip install -q --upgrade tensorflow-gpu==2.0.0-alpha0
  print('Installation of `tensorflow-gpu==2.0.0-alpha0` complete.')
elif TF_Installation == 'TF1 Nightly (GPU)':
  !pip install -q --upgrade tf-nightly-gpu
  print('Installation of `tf-nightly-gpu` complete.')
elif TF_Installation == 'TF1 Stable (GPU)':
  !pip install -q --upgrade tensorflow-gpu
  print('Installation of `tensorflow-gpu` complete.')
elif TF_Installation == 'System':
  pass
else:
  raise ValueError('Selection Error: Please select a valid '
                   'installation option.')

     |████████████████████████████████| 369.9MB 49kB/s 
     |████████████████████████████████| 61kB 27.9MB/s 
     |████████████████████████████████| 450kB 56.5MB/s 
     |████████████████████████████████| 3.2MB 50.4MB/s 
Installation of `tf-nightly-gpu-2.0-preview` complete.


In [0]:
#@title Import { display-mode: "form" }

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np

import tensorflow as tf
from tensorflow.python import tf2
if not tf2.enabled():
  import tensorflow.compat.v2 as tf
  tf.enable_v2_behavior()
  assert tf2.enabled()

import tensorflow_datasets as tfds
import tensorflow_probability as tfp


tfk = tf.keras
tfkl = tf.keras.layers
tfpl = tfp.layers
tfd = tfp.distributions

In [3]:
if tf.test.gpu_device_name() != '/device:GPU:0':
  print('WARNING: GPU device not found.')
else:
  print('SUCCESS: Found GPU: {}'.format(tf.test.gpu_device_name()))

SUCCESS: Found GPU: /device:GPU:0


In [0]:
# #Step 1: import dependencies
# from tensorflow.keras import layers
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# import tensorflow as tf
# from keras import regularizers
# import time
# from __future__ import division

# import tensorflow as tf
# import tensorflow_probability as tfp
# tfd = tfp.distributions

# %matplotlib inline
# plt.style.use('ggplot')

In [0]:
#Multivariate diagional
def create_encoder(inpt, latent_size):
  x = inpt
  x = tf.layers.dense(x, 64, tf.nn.relu)
  x = tf.layers.dense(x, 32, tf.nn.relu)
  x = tf.layers.dense(x, 16, tf.nn.relu)
  loc = tf.layers.dense(x,latent_size)
  log_variance = tf.layers.dense(x, latent_size)
  scale = tf.nn.softplus(log_variance)
  return tfd.MultivariateNormalDiag(loc, scale)

def create_prior(latent_size):
  return tfd.Independent(tfd.Normal(loc=tf.zeros(latent_size), scale=1), reinterpreted_batch_ndims=1)

def create_decoder(inpt, output_size):
  x = inpt
  x = tf.layers.dense(x, 16, tf.nn.relu)
  x = tf.layers.dense(x, 32, tf.nn.relu)
  x = tf.layers.dense(x, 64, tf.nn.relu)
  loc = tf.layers.dense(x, output_size)
  scale = 0.0001*tf.ones(output_size)
  return tfd.MultivariateNormalDiag(loc, scale)

In [0]:
#Connect encoder and decoder and define the loss function
tf.reset_default_graph()

x = tf.placeholder(tf.float32, shape=[None, N_genes], name='x')

latent_prior = create_prior(latent_size=2)

print(latent_prior.batch_shape)
print(latent_prior.event_shape)
latent_posterior = create_encoder(x, latent_size=2)

print(latent_posterior.batch_shape)
print(latent_posterior.event_shape)

latent_sampled = latent_posterior.sample()
latent_mean = latent_posterior.mean()

posterior = create_decoder(latent_sampled, X.shape[1])

print(posterior.batch_shape)
print(posterior.event_shape)

posterior_sampled = posterior.sample()
posterior_mean = posterior.mean()

rec_loss = -tf.reduce_mean(posterior.log_prob(x))
kl = tf.reduce_mean(tfd.kl_divergence(latent_posterior, latent_prior)) #analytical
#kl = tf.reduce_mean(latent_posterior.log_prob(latent_sampled)- latent_prior.log_prob(latent_sampled)) #approximate KL
beta=0.000000001
gamma = 10.0
#kl_loss = beta*kl
#C = tf.placeholder(tf.float32, name='C') 
kl_loss = beta*kl
loss = rec_loss + kl_loss
optimizer = tf.train.AdamOptimizer(0.001).minimize(loss)

In [0]:
input_shape = datasets_info.features['image'].shape
encoded_size = 16
base_depth = 32

In [0]:
prior = tfd.Independent(tfd.Normal(loc=tf.zeros(encoded_size), scale=1),
                        reinterpreted_batch_ndims=1)